In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [4]:
from nlpsig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [5]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [6]:
%run ../load_anno_mi.py

In [7]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-06 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-06 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-06 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-06 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-06 00:00:34,-1


In [8]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

## Baseline: Fine-tune BERT for classification

In [9]:
num_epochs = 5
seeds = [0, 1, 12, 123, 1234]
validation_metric = "f1"

In [10]:
label_to_id_client

{'neutral': 0, 'change': 1, 'sustain': 2}

In [11]:
id_to_label_client

{0: 'neutral', 1: 'change', 2: 'sustain'}

In [12]:
kwargs = {
    "num_epochs": num_epochs,
    "pretrained_model_name": "bert-base-uncased",
    "df": anno_mi,
    "feature_name": "utterance_text",
    "label_column": "client_talk_type",
    "label_to_id": label_to_id_client,
    "id_to_label": id_to_label_client,
    "output_dim": output_dim_client,
    "seeds": seeds,
    "device": device,
    "batch_size": 8,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "validation_metric": validation_metric,
    "verbose": False,
}

## Focal Loss

In [13]:
loss = "focal"
gamma = 2

In [14]:
bert_classifier = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_focal.csv",
    **kwargs,
)

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.675592,0.659985,0.416835
2,0.669000,0.635145,0.616808,0.522742
3,0.669000,1.108824,0.639938,0.487981
4,0.478000,1.217864,0.663840,0.530595
5,0.186100,1.401290,0.667695,0.528114


Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.655853,0.680522,0.391569
2,0.635700,0.679611,0.623472,0.515699
3,0.635700,1.057101,0.661777,0.540569
4,0.376600,1.346776,0.661777,0.525447
5,0.164700,1.454775,0.665037,0.530367


saving the results dataframe to CSV in client_talk_type_output/bert_classifier_focal.csv


In [15]:
bert_classifier

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.620306,0.510270,"[0.7431776834445118, 0.44533120510774143, 0.34...",0.500967,"[0.7810707456978967, 0.4312210200927357, 0.290...",0.528517,"[0.7087912087912088, 0.4603960396039604, 0.416...",1,focal,2,True
0,0.583525,0.468548,"[0.7247805328815647, 0.37351703023344807, 0.30...",0.461697,"[0.7752883031301483, 0.3483226266952177, 0.261...",0.485273,"[0.6804511278195489, 0.40264026402640263, 0.37...",12,focal,2,True
0,0.620690,0.403734,"[0.7567351293678315, 0.31513537505548156, 0.13...",0.462857,"[0.7022277227722772, 0.3410182516810759, 0.345...",0.400198,"[0.8204164256795836, 0.2929042904290429, 0.087...",123,focal,2,True


In [16]:
bert_classifier["f1"].mean()

0.4608508435414917

In [17]:
bert_classifier["precision"].mean()

0.4751734599965687

In [18]:
bert_classifier["recall"].mean()

0.4713292214125982

In [19]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

array([0.74156445, 0.37799454, 0.26299355])

In [20]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

array([0.75286226, 0.37352063, 0.29913749])

In [21]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)

array([0.73655292, 0.38531353, 0.29212121])

## Using Cross-Entropy loss

In [14]:
loss = "cross_entropy"
gamma = None

In [15]:
bert_classifier_ce = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_ce.csv",
    **kwargs,
)

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.785765,0.672321,0.432937
2,0.775400,0.776554,0.683115,0.534713
3,0.775400,1.093161,0.667695,0.505132
4,0.489000,1.485090,0.673863,0.551776
5,0.232900,1.669982,0.670008,0.553317


Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.768551,0.683782,0.377543
2,0.741100,0.767284,0.682152,0.545899
3,0.741100,1.167989,0.656887,0.547341
4,0.437500,1.648849,0.661777,0.541317
5,0.213800,1.690377,0.678892,0.536598


saving the results dataframe to CSV in client_talk_type_output/bert_classifier_ce.csv


In [16]:
bert_classifier_ce

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.667816,0.482975,"[0.7897202516396734, 0.39833641404805914, 0.26...",0.533202,"[0.7351108896087715, 0.4527310924369748, 0.411...",0.466538,"[0.8530942741469058, 0.3556105610561056, 0.190...",1,cross_entropy,None,True
0,0.653065,0.471073,"[0.7802614930409111, 0.4383346425765907, 0.194...",0.502176,"[0.759233926128591, 0.41829085457271364, 0.329...",0.467022,"[0.8024869866975131, 0.4603960396039604, 0.138...",12,cross_entropy,None,True
0,0.658812,0.493624,"[0.7851562500000001, 0.4317984361424848, 0.263...",0.506405,"[0.7584905660377359, 0.4559633027522936, 0.304...",0.485519,"[0.8137651821862348, 0.41006600660066006, 0.23...",123,cross_entropy,None,True


In [17]:
bert_classifier_ce["f1"].mean()

0.48255742861862533

In [18]:
bert_classifier_ce["precision"].mean()

0.513927952353963

In [19]:
bert_classifier_ce["recall"].mean()

0.4730263591232846

In [20]:
np.stack(bert_classifier_ce["f1_scores"]).mean(axis=0)

array([0.785046  , 0.42282316, 0.23980312])

In [21]:
np.stack(bert_classifier_ce["precision_scores"]).mean(axis=0)

array([0.75094513, 0.44232842, 0.34851031])

In [22]:
np.stack(bert_classifier_ce["recall_scores"]).mean(axis=0)

array([0.82311548, 0.40869087, 0.18727273])